## Load Packages

In [6]:
#Import libraries and set random seed
import pandas as pd
import numpy as np
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from collections import Counter
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix


np.random.seed(42)

## Olink Data

In [7]:
#Read In Olink Data, Format, and Find Percent Missing
ukolink = pd.read_csv("/home/ffc4001/ukbiobankdata/olink_data.txt",sep='\t')
ukolink1 = ukolink[ukolink["ins_index"] == 0]
ukolink1.drop(["ins_index"], axis=1, inplace=True)
ukolink1 = ukolink1.pivot_table(values='result', columns='protein_id', index = "eid", aggfunc='first')
percent_missing = ukolink1.isnull().sum() * 100 / len(ukolink1)
missing_value_df = pd.DataFrame({'column_name': ukolink1.columns,
                                 'percent_missing': percent_missing})

/tmp/ipykernel_793386/959808266.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ukolink1.drop(["ins_index"], axis=1, inplace=True)


In [8]:
#Show Percent Missing
missing_value_df.sort_values('percent_missing')

,column_name,percent_missing
protein_id,,
619,619,1.017001
1881,1881,1.017001
361,361,1.017001
2397,2397,1.017001
2381,2381,1.017001
...,...,...
115,115,8.860808
2614,2614,13.378491
732,732,23.170917


In [10]:
#Replace uniprot_id's with protein names
olinknames = pd.read_csv("/home/ffc4001/ukbiobankdata/Data/coding143.tsv",sep='\t')
olinknames_dict = dict(zip(olinknames.coding, olinknames.meaning))

In [103]:
olinknames_dict[2297]

'RNASET2;Ribonuclease T2'

In [11]:
ukolink1.columns = ukolink1.columns.map(olinknames_dict)

In [12]:
ukolink1

protein_id,AARSD1;Alanyl-tRNA editing protein Aarsd1,ABHD14B;Protein ABHD14B,ABL1;Tyrosine-protein kinase ABL1,"ACAA1;3-ketoacyl-CoA thiolase, peroxisomal",ACAN;Aggrecan core protein,ACE2;Angiotensin-converting enzyme 2,ACOX1;Peroxisomal acyl-coenzyme A oxidase 1,ACP5;Tartrate-resistant acid phosphatase type 5,ACP6;Lysophosphatidic acid phosphatase type 6,"ACTA2;Actin, aortic smooth muscle",...,WNT9A;Protein Wnt-9a,WWP2;NEDD4-like E3 ubiquitin-protein ligase WWP2,XCL1;Lymphotactin,XG;Glycoprotein Xg,XPNPEP2;Xaa-Pro aminopeptidase 2,XRCC4;DNA repair protein XRCC4,YES1;Tyrosine-protein kinase Yes,YTHDF3;YTH domain-containing family protein 3,ZBTB16;Zinc finger and BTB domain-containing protein 16,ZBTB17;Zinc finger and BTB domain-containing protein 17
eid,,,,,,,,,,,,,,,,,,,,,
1000048,-0.44185,NaN,-0.37680,1.83095,NaN,NaN,NaN,NaN,-0.49450,NaN,...,0.9734,NaN,1.35320,NaN,-1.32800,NaN,-0.7925,-1.18150,-0.58535,NaN
1000072,0.26705,0.19705,1.36915,0.89980,0.45650,1.75230,0.31135,0.24905,2.10985,0.68200,...,0.2045,0.9581,-0.58690,-0.25035,0.35035,-0.34695,2.3289,1.49335,1.39360,-0.13565
1000256,-0.08155,0.23025,0.16140,-0.65685,0.06830,-0.08315,0.16710,-0.43970,0.54610,0.42175,...,0.4965,-0.2325,-0.50680,0.23510,0.03730,-0.06620,0.5781,0.60500,-0.43985,-0.13210
1000305,-0.29995,0.34875,0.53075,2.46510,-0.17530,0.89800,1.50775,-0.02125,-0.58265,0.63660,...,-0.2612,0.5441,-0.77760,0.15825,-0.94215,0.37285,0.9376,0.11665,0.65120,-0.09725
1000364,0.15620,-0.72080,-0.48390,-1.04680,-0.16280,-0.09300,0.01440,-0.25280,0.10330,-0.02770,...,0.1755,-0.6188,-0.59860,-0.42140,0.76950,-0.42335,-0.9590,-1.47930,-0.26205,-0.25430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6024705,-0.73905,0.36930,-0.14390,1.50760,0.21720,0.28195,-0.43570,-0.10540,-0.14640,0.15380,...,NaN,-0.6455,-0.31160,0.54810,1.25920,-0.07475,-1.5997,NaN,-0.83605,0.28210
6024773,-0.11700,-0.02110,-1.09680,1.79300,0.25010,0.32895,0.29840,0.07500,0.09850,0.53850,...,0.1482,-1.0865,-0.42150,-0.35830,-2.50120,0.31715,-3.6805,-0.49145,-0.78885,0.31050
6025049,-0.61460,0.60790,0.11420,-0.52910,0.51750,-0.28175,-0.27260,0.10640,-0.63840,0.20410,...,0.5065,0.0827,0.03530,0.71250,-0.14010,1.08485,0.0000,0.98685,-0.51745,1.13160


In [24]:
eid = ukolink1.index
#eid.to_csv("/home/ffc4001/ukbiobankdata/Data/olink_eid.csv")

In [ ]:
# Create correlation matrix WORK IN PROGRESS
corr_matrix = df.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

# Drop features 
df.drop(to_drop, axis=1, inplace=True)

## Read in Variant and Demographic Data

In [28]:
#Read in Variant only data
Trn_input =  pd.read_csv("/home/ffc4001/ukbiobankdata/TrainTest/ukbiobank_variantsonlytraindata_eid.csv")
Trn_target = pd.read_csv("/home/ffc4001/ukbiobankdata/TrainTest/ukbiobank_variantsonlytrainy_eid.csv")
Test_input = pd.read_csv("/home/ffc4001/ukbiobankdata/TrainTest/ukbiobank_variantsonlytestdata_eid.csv")
Test_target = pd.read_csv("/home/ffc4001/ukbiobankdata/TrainTest/ukbiobank_variantsonlytesty_eid.csv")
Val_input = pd.read_csv("/home/ffc4001/ukbiobankdata/TrainTest/ppmi_variantsonlyvaldata_eid.csv")
Val_target = pd.read_csv("/home/ffc4001/ukbiobankdata/TrainTest/ppmi_variantsonlyvaly_eid.csv")
Trn_input = pd.concat([Trn_input, pd.DataFrame(Trn_target)], axis=1)
Test_input = pd.concat([Test_input, pd.DataFrame(Test_target)], axis=1)

In [29]:
Trn_input.drop(["Unnamed: 0"], axis=1, inplace=True) 
Trn_target.drop(["Unnamed: 0"], axis=1, inplace=True) 
Test_input.drop(["Unnamed: 0"], axis=1, inplace=True)
Test_target.drop(["Unnamed: 0"], axis=1, inplace=True) 
Val_input.drop(["Unnamed: 0"], axis=1, inplace=True)
Val_target.drop(["Unnamed: 0"], axis=1, inplace=True) 

In [30]:
ukolink1.index.names = ['eid_correct']
ukblood_values = ukolink1

In [31]:
#Merge Olink Data to PPMI Data
Trn_input = Trn_input.merge(ukblood_values,how='inner', on="eid_correct")
Test_input = Test_input.merge(ukblood_values,how='inner', on="eid_correct")

In [32]:
Trn_target = Trn_input["Y"]
Test_target = Test_input["Y"]

In [33]:
Trn_input.drop(["Y"], axis=1, inplace=True)
Test_input.drop(["Y"], axis=1, inplace=True)

In [48]:
Trn_input.drop(["eid_correct"], axis=1, inplace=True)
Test_input.drop(["eid_correct"], axis=1, inplace=True)

In [49]:
#Imputation
Trn_input.fillna(Trn_input.mode().iloc[0], inplace=True)
Test_input.fillna(Test_input.mode().iloc[0], inplace=True)

In [ ]:
#age plus olink normalization
column = "age"

In [ ]:
# #Standarize age, and biomarkers
# Trn_input[column] = (Trn_input[column] - Trn_input[column].min()) / (Trn_input[column].max() - Trn_input[column].min())
# Test_input[column] = (Test_input[column] - Test_input[column].min()) / (Test_input[column].max() - Test_input[column].min())
# X_val[column] = (X_val[column] - X_val[column].min()) / (X_val[column].max() - X_val[column].min())

In [45]:
#eid3 = pd.concat([eid_correct1, eid_correct2])

In [47]:
#eid3.to_csv("/home/ffc4001/ukbiobankdata/Data/olink_eid.csv")

## Add Olink to PPMI and align to UK Biobank.

In [50]:
X_val = pd.read_pickle("/home/ffc4001/ukbiobankdata/Data/final_X_valvariantsonly")


In [532]:
Y_val = pd.read_pickle("/home/ffc4001/ukbiobankdata/Data/final_Y_valvariantsonly")

In [58]:
ppmi_id = Val_input[["eid_correct"]]

In [60]:
X_val["PATNO"] = ppmi_id["eid_correct"]

In [472]:
olink_cardio_ppmi = pd.read_csv("/home/ffc4001/ukbiobankdata/Data/PPMI_Project_196_Plasma_Cardio_NPX_13Dec2023.csv")
olink_immune_ppmi = pd.read_csv("/home/ffc4001/ukbiobankdata/Data/PPMI_Project_196_Plasma_INF_NPX_13Dec2023.csv")
olink_cancer_ppmi = pd.read_csv("/home/ffc4001/ukbiobankdata/Data/PPMI_Project_196_Plasma_ONC_NPX_13Dec2023.csv")
olink_neuro_ppmi = pd.read_csv("/home/ffc4001/ukbiobankdata/Data/PPMI_Project_196_Plasma_NEURO_NPX_13Dec2023.csv")

In [480]:
olink_immune_ppmi = olink_immune_ppmi.loc[olink_immune_ppmi['EVENT_ID'] == "BL"]
olink_cardio_ppmi = olink_cardio_ppmi.loc[olink_cardio_ppmi['EVENT_ID'] == "BL"]
olink_cancer_ppmi = olink_cancer_ppmi.loc[olink_cancer_ppmi['EVENT_ID'] == "BL"]
olink_neuro_ppmi = olink_neuro_ppmi.loc[olink_neuro_ppmi['EVENT_ID'] == "BL"]

In [476]:
len(bl["PATNO"].unique())

227

In [437]:
#temp2 = olink_cardio_ppmi[olink_cardio_ppmi["PATNO"] == "PPMI-3907"]

In [ ]:
#Match to ukbiobank column name
#I need patno by NPX with column names ASSAY

In [125]:
#olink_neuro_ppmi['PATNO']. nunique()

295

In [481]:
olink_cardio_ppmi1 = olink_cardio_ppmi[["PATNO", "ASSAY", "NPX"]]
olink_immune_ppmi1 = olink_immune_ppmi[["PATNO", "ASSAY", "NPX"]]
olink_neuro_ppmi1 = olink_neuro_ppmi[["PATNO", "ASSAY", "NPX"]]
olink_cancer_ppmi1 = olink_cancer_ppmi[["PATNO", "ASSAY", "NPX"]]

In [ ]:
#Take first visit each patno

In [ ]:
#Drop ST
#RECODE 0:15 bL to V10


In [410]:
#event_id_dict = {"BL":0, "V01":1,"V02":2,"V03":3,"V04":4,"V05":5,"V06":6,"V07":7,"V08":8,"V09":9,"V10":10,"V11":11,"V12":12,"V13":13,"V14":14,"V15":15, "ST":16}



In [411]:
#olink_cardio_ppmi1=olink_cardio_ppmi1.replace({"EVENT_ID": event_id_dict})

In [416]:
#olink_cardio_ppmi1["EVENT_ID"] = pd.to_numeric(olink_cardio_ppmi1["EVENT_ID"])

In [421]:
#olink_cardio_ppmi1["EVENT_ID"].fillna(17, inplace=True)

In [422]:
#olink_cardio_ppmi1["EVENT_ID"].isna().sum()

0

In [435]:
#temp1 = olink_cardio_ppmi1.loc[olink_cardio_ppmi1.groupby('PATNO')['EVENT_ID'].idxmin()]

In [485]:
olink_cardio_ppmi2 = olink_cardio_ppmi1.pivot_table(['NPX'], 'PATNO','ASSAY')
olink_neuro_ppmi2 = olink_neuro_ppmi1.pivot_table(['NPX'], 'PATNO','ASSAY')
olink_cancer_ppmi2 = olink_cancer_ppmi1.pivot_table(['NPX'], 'PATNO','ASSAY')
olink_immune_ppmi2 = olink_immune_ppmi1.pivot_table(['NPX'], 'PATNO','ASSAY')

In [489]:
olink_neuro_ppmi2.columns = olink_neuro_ppmi2.columns.droplevel()

In [490]:
olink_cancer_ppmi2.columns = olink_cancer_ppmi2.columns.droplevel()
olink_immune_ppmi2.columns = olink_immune_ppmi2.columns.droplevel()
olink_cardio_ppmi2.columns = olink_cardio_ppmi2.columns.droplevel()

In [491]:
olink_cancer_ppmi2.shape

(190, 370)

In [493]:
#merge data 
merge = pd.merge(olink_cancer_ppmi2, olink_neuro_ppmi2, on="PATNO") 
merge = pd.merge(merge, olink_immune_ppmi2, on="PATNO")
#merge = pd.merge(merge, olink_cardio_ppmi2, on="PATNO")


In [494]:
merge = pd.merge(merge, olink_cardio_ppmi2, on="PATNO")

/tmp/ipykernel_793386/3976361491.py:1: FutureWarning: Passing 'suffixes' which cause duplicate columns {'TNF_x', 'IL6_x', 'CXCL8_x'} in the result is deprecated and will raise a MergeError in a future version.
  merge = pd.merge(merge, olink_cardio_ppmi2, on="PATNO")


In [495]:
merge.columns.unique()
#DUPLICATES?

Index(['AARSD1', 'ABL1', 'ACAA1', 'ACP6', 'ADAMTS15', 'ADAMTS8', 'ADCYAP1R1',
       'ADGRG1', 'ADM', 'AGR3',
       ...
       'VAMP5', 'VASN', 'VCAM1', 'VIM', 'VSIR', 'VSTM2L', 'VWF', 'WASF1', 'XG',
       'ZBTB17'],
      dtype='object', name='ASSAY', length=1475)

In [497]:
merge['id'] = merge.index.str.split('-').str[1]

In [196]:
#merge.drop(merge.tail(2).index,
 #       inplace = True)

In [505]:
merge['id'] = merge['id'].astype(np.float64)

In [506]:
merge["PATNO"] = merge['id']
merge.index.names = ['id1']

In [509]:
X_val1 = pd.merge(X_val, merge, on="PATNO") 

In [511]:
X_val1[["id"]].to_csv("X_val_PATNO.csv")

In [513]:
Y_val["PATNO"] = X_val["PATNO"]

In [534]:
Y_val1 = pd.DataFrame(Y_val.T, columns=['Y'])

In [536]:
Y_val1["PATNO"] = X_val["PATNO"]

In [538]:
Y_val1 = pd.merge(Y_val1, X_val1["PATNO"],  on="PATNO")

In [540]:
Y_val1 = Y_val1["Y"]

In [541]:
Y_val1.sum()

26.0

In [320]:
Y_val1

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
124    0.0
125    1.0
126    1.0
127    1.0
128    1.0
Name: Y, Length: 129, dtype: float64

In [ ]:
#Align olink data.

In [543]:
column_names = Trn_input.columns[637:]

In [545]:
old_names = Trn_input.columns[637:]
Trn_input.rename(columns=dict(zip(old_names, old_names.str.split(';').str[0])), inplace=True)

In [544]:
old_names = Test_input.columns[637:]
Test_input.rename(columns=dict(zip(old_names, old_names.str.split(';').str[0])), inplace=True)

In [577]:
age_info = pd.concat([Trn_input, Test_input], axis=0)

In [583]:
#Min, Max, Mediian, Mean
age_info.Age.max()

70

In [586]:
age_info.Age.min()

40

In [585]:
age_info.Age.mean()

57.26322336896425

In [584]:
age_info.Age.median()

59.0

In [ ]:
#Min, Max, Mediian, Mean

In [549]:
names = Trn_input.columns

In [550]:
names

Index(['Inferred_Gender', 'Age', 'X1.8495945_T_C_C', 'X1.20149058_A_G_G',
       'X1.20149560_G_A_A', 'X1.20150839_A_G_G', 'X1.20150984_C_G_G',
       'X1.20150998_A_G_G', 'X1.20151290_C_A_A', 'X1.20151431_A_G_G',
       ...
       'WNT9A', 'WWP2', 'XCL1', 'XG', 'XPNPEP2', 'XRCC4', 'YES1', 'YTHDF3',
       'ZBTB16', 'ZBTB17'],
      dtype='object', length=2100)

In [552]:
X_val1.columns.intersection(names)

Index(['Age', 'Inferred_Gender', 'X1.108176287_C_T_T', 'X1.151115007_G_A_A',
       'X1.154837939_A_G_G', 'X1.155026114_C_T_C', 'X1.155033317_T_C_T',
       'X1.155033918_C_T_T', 'X1.155034632_C_G_G', 'X1.155044197_T_G_G',
       ...
       'VAMP5', 'VASN', 'VCAM1', 'VIM', 'VSIR', 'VSTM2L', 'VWF', 'WASF1', 'XG',
       'ZBTB17'],
      dtype='object', length=2003)

In [553]:
X_val2 = X_val1[X_val1.columns.intersection(names)]

In [555]:
Trn_input1 = Trn_input[Trn_input.columns.intersection(X_val2.columns)]

In [556]:
Test_input1 = Test_input[Test_input.columns.intersection(X_val2.columns)]

In [558]:
#standardize age and move inferred gender to front for ppmi
column = "Age"

In [559]:
#Standarize age, and biomarkers
Trn_input1[column] = (Trn_input1[column] - Trn_input1[column].min()) / (Trn_input1[column].max() - Trn_input1[column].min())
Test_input1[column] = (Test_input1[column] - Test_input1[column].min()) / (Test_input1[column].max() - Test_input1[column].min())
#X_val[column] = (X_val[column] - X_val[column].min()) / (X_val[column].max() - X_val[column].min())

/tmp/ipykernel_793386/2651736735.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Trn_input1[column] = (Trn_input1[column] - Trn_input1[column].min()) / (Trn_input1[column].max() - Trn_input1[column].min())
/tmp/ipykernel_793386/2651736735.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Test_input1[column] = (Test_input1[column] - Test_input1[column].min()) / (Test_input1[column].max() - Test_input1[column].min())


In [ ]:
Test_input.iloc[1:2,0:638]

In [646]:
sample10 = Test_input.iloc[:, 637:]

In [648]:
sample10.columns

Index(['AARSD1', 'ABHD14B', 'ABL1', 'ACAA1', 'ACAN', 'ACE2', 'ACOX1', 'ACP5',
       'ACP6', 'ACTA2',
       ...
       'WNT9A', 'WWP2', 'XCL1', 'XG', 'XPNPEP2', 'XRCC4', 'YES1', 'YTHDF3',
       'ZBTB16', 'ZBTB17'],
      dtype='object', length=1463)

In [561]:
#Save TRAIN, TEST, VAL

In [564]:
first_column = X_val2.pop('Inferred_Gender') 
X_val2.insert(0,'Inferred_Gender', first_column)

In [303]:
#X_val2.fillna(X_val2.mode().iloc[0], inplace=True)

/tmp/ipykernel_793386/3911273629.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val2.fillna(Trn_input.mode().iloc[0], inplace=True)


In [294]:
# #Save TRAIN, TEST, VAL

# Trn_input1.to_pickle("/home/ffc4001/ukbiobankdata/Data/final_X_trainolink")
# Trn_target.to_pickle("/home/ffc4001/ukbiobankdata/Data/final_Y_trainolink")
# Test_input1.to_pickle("/home/ffc4001/ukbiobankdata/Data/final_X_testolink")
# Test_target.to_pickle("/home/ffc4001/ukbiobankdata/Data/final_Y_testolink")

In [568]:
combined_ppmi_uk_target = pd.concat([Test_target, Y_val1])

In [569]:
combined_ppmi_uk_target_input = pd.concat([Test_input1, X_val2])

In [305]:
# combined_ppmi_uk_target_input.to_pickle("/home/ffc4001/ukbbl)
# combined_ppmi_uk_target.to_pickle("/home/ffc4001/ukbiobankdata/Data/final_Y_combinedtest")

In [571]:
x_val2 = X_val2[Test_input1.columns]
x_val2.fillna(x_val2.mode().iloc[0], inplace=True)

In [572]:
#x_val2.to_pickle("/home/ffc4001/ukbiobankdata/Data/final_updated_X_valolink")
#Y_val1.to_pickle("/home/ffc4001/ukbiobankdata/Data/final_updated_Y_valolink")

In [573]:
temp3 = pd.read_pickle("/home/ffc4001/ukbiobankdata/Data/final_combinedtest")

In [575]:
#temp3.columns.intersection(x_val2.columns)

Index(['Inferred_Gender', 'Age', 'X1.8495945_T_C_C', 'X1.20149058_A_G_G',
       'X1.20149560_G_A_A', 'X1.20150839_A_G_G', 'X1.20150984_C_G_G',
       'X1.20150998_A_G_G', 'X1.20151290_C_A_A', 'X1.20151431_A_G_G',
       ...
       'WNT9A', 'WWP2', 'XCL1', 'XG', 'XPNPEP2', 'XRCC4', 'YES1', 'YTHDF3',
       'ZBTB16', 'ZBTB17'],
      dtype='object', length=2003)

In [ ]:
#make ppmi updrs score training test


In [ ]:
#Seperate ppmi to test
#improve variant neural network
#Figures

In [319]:
#!jupyter nbconvert --to html Preprocessing_olink.ipynb

[NbConvertApp] Converting notebook Preprocessing_olink.ipynb to html
[NbConvertApp] Writing 889843 bytes to Preprocessing_olink.html


In [591]:
demographics_ppmi = pd.concat([ppmi_id, Val_input], axis=1)

In [616]:
demographics_ppmi["eid_correct1"] =demographics_ppmi["eid_correct"].iloc[:, 0]

In [619]:
demo_ppmi = demographics_ppmi[demographics_ppmi.eid_correct1.isin(X_val1.PATNO)]

In [621]:
demo_ppmi.Age.max()

82.712328767

In [625]:
demo_ppmi.Age.min()

30.615824538

In [626]:
demo_ppmi.Age.mean()

60.41142628107766

In [627]:
demo_ppmi.Age.median()

60.454083389

In [628]:
demo_ppmi.Inferred_Gender.value_counts()

1.0    74
0.0    29
Name: Inferred_Gender, dtype: int64

In [ ]:
#ratio of features case control for uk bio

In [ ]:
#Trn_input1.to_pickle("/home/ffc4001/ukbiobankdata/Data/final_X_trainolink")
# Trn_target.to_pickle("/home/ffc4001/ukbiobankdata/Data/final_Y_trainolink")
# Test_input1.to_pickle("/home/ffc4001/ukbiobankdata/Data/final_X_testolink")
# Test_target.to_pickle("/home/ffc4001/ukbiobankdata/Data/final_Y_testolink")

In [652]:
Trn_input5 = pd.concat([Trn_input1, pd.DataFrame(Trn_target)], axis=1)

In [654]:
Test_input5 = pd.concat([Test_input1, pd.DataFrame(Test_target)], axis=1)

In [666]:
combined_5 = pd.concat([Trn_input5, Test_input5])

In [664]:
combined_6 = combined_5.groupby('Y', as_index=False).agg(median = ('X1.8495945_T_C_C','median'), mean = ('X1.8495945_T_C_C','mean'))

In [665]:
combined_6

,Y,median,mean
0,0.0,1.0,0.686887
1,1.0,1.0,0.677378


In [668]:
combined_5.groupby('Y', as_index=False).mean()

/tmp/ipykernel_793386/2513154157.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_5.groupby('Y', as_index=False).mean()


,Y,Inferred_Gender,Age,X1.8495945_T_C_C,X1.20149058_A_G_G,X1.20149560_G_A_A,X1.20150839_A_G_G,X1.20150984_C_G_G,X1.20150998_A_G_G,X1.20151290_C_A_A,...,WNT9A,WWP2,XCL1,XG,XPNPEP2,XRCC4,YES1,YTHDF3,ZBTB16,ZBTB17
0,0.0,0.459606,0.571830,0.686887,0.492189,0.525874,0.531316,0.531058,0.531081,0.497631,...,0.020122,0.023118,0.056523,0.014185,-0.13119,0.018173,-0.021877,0.017250,0.112463,0.044606
1,1.0,0.624679,0.773265,0.677378,0.466581,0.497429,0.498715,0.498715,0.498715,0.473008,...,0.134582,-0.016600,0.111749,-0.056965,-0.30168,0.007238,-0.031092,-0.025293,0.149813,0.043117


In [669]:
combined_5.groupby('Y', as_index=False).median()

/tmp/ipykernel_793386/760516045.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_5.groupby('Y', as_index=False).median()


,Y,Inferred_Gender,Age,X1.8495945_T_C_C,X1.20149058_A_G_G,X1.20149560_G_A_A,X1.20150839_A_G_G,X1.20150984_C_G_G,X1.20150998_A_G_G,X1.20151290_C_A_A,...,WNT9A,WWP2,XCL1,XG,XPNPEP2,XRCC4,YES1,YTHDF3,ZBTB16,ZBTB17
0,0.0,0.0,0.633333,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0000,0.00000,0.0035,-0.000100,0.000000,-0.00075,0.0,0.0,-0.00145,-0.0031
1,1.0,1.0,0.833333,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.1176,-0.03075,0.0043,-0.094075,-0.164175,-0.00075,0.0,0.0,0.00830,-0.0031


## T-test on Biomarkers

In [ ]:
# get a list of all columns in the dataframe without the Group column
column_list = [x for x in combined_5.columns if x != 'Y']
# create an empty dictionary
t_test_results = {}
# loop over column_list and execute code explained above
for column in column_list:
    Control = combined_5.where(combined_5.Y== 0).dropna()[column]
    Case = combined_5.where(combined_5.Y== 1).dropna()[column]
    # add the output to the dictionary 
    t_test_results[column] = scipy.stats.ttest_ind(Case,Control)
results_df = pd.DataFrame.from_dict(t_test_results,orient='Index')
results_df.columns = ['statistic','pvalue']

In [674]:
import scipy
#MAKE INTO VOLCANO PLOT

In [708]:
results_df.iloc[1830:1860,]

,statistic,pvalue
SRP14,0.413849,6.789870e-01
SRPK2,0.415373,6.778709e-01
SSB,-1.134794,2.564678e-01
SSC4D,-0.878311,3.797800e-01
SSC5D,3.470180,5.206145e-04
ST3GAL1,-1.417707,1.562834e-01
ST6GAL1,-1.583644,1.132820e-01
STAMBP,0.220733,8.253015e-01
STAT5B,-0.581764,5.607286e-01
STC1,2.083864,3.717840e-02


In [711]:
results_df.loc[["PAEP", "GH1", "IGFBP1", "IL18RAP", "SIGLEC5", "CGA","XPNPEP2", "MUC16", "CCL19", "SCGB3A2","PSG1","PRL","TNNI3","CRH","CDHR2","IL5","GH2","FABP9","SNCG","FABP4"]]

,statistic,pvalue
PAEP,-5.649376,1.620369e-08
GH1,1.519422,1.286635e-01
IGFBP1,-1.309917,1.902310e-01
IL18RAP,-3.296841,9.785717e-04
SIGLEC5,2.735318,6.234530e-03
CGA,-1.218722,2.229562e-01
XPNPEP2,-4.288065,1.806248e-05
MUC16,1.297263,1.945475e-01
CCL19,2.950052,3.178900e-03
SCGB3A2,-1.049186,2.940983e-01


In [722]:
print(results_df.sort_values(['statistic'], ascending=[False]).to_string())

                     statistic        pvalue
Age                  20.733913  4.955583e-95
NEFL                 15.645333  5.055847e-55
EDA2R                14.105700  4.407103e-45
GDF15                12.983077  1.800313e-38
LTBP2                10.439550  1.753647e-25
HSPB6                10.314151  6.498462e-25
TSPAN1                9.985084  1.877870e-23
CCN5                  9.941950  2.895502e-23
PGF                   9.698728  3.215647e-22
NCS1                  9.482492  2.603674e-21
CDCP1                 9.463667  3.116875e-21
MERTK                 9.427470  4.400724e-21
WNT9A                 9.223796  2.992482e-20
Inferred_Gender       9.160053  5.406744e-20
SCARF2                9.062634  1.325008e-19
CD276                 9.041983  1.600353e-19
EBI3_IL27             9.012573  2.092578e-19
ADGRG1                8.746775  2.272547e-18
HAVCR1                8.369172  5.973347e-17
ACTA2                 8.266988  1.412378e-16
RSPO3                 8.225365  1.999455e-16
NTproBNP  

In [ ]:
CST3                  5.066854  4.061001e-07   + Y
DKK3                  5.555535  2.783826e-08   - N
FABP5                -1.863905  6.234182e-02   + N
GRN                  -0.685891  4.927856e-01   - Y
ICAM1                 0.183723  8.542315e-01   
PTGDS                 5.829303  5.605208e-09   + Y
SELE                 -1.499802  1.336730e-01   - Y
VCAM1                 6.710640  1.961481e-11   + Y
ENO1                  0.075367  9.399234e-01   


In [ ]:
CST3                  5.066854  4.061001e-07   + Y
DKK3                  5.555535  2.783826e-08   - N
FABP5                -1.863905  6.234182e-02   + N
GRN                  -0.685891  4.927856e-01   - Y
PTGDS                 5.829303  5.605208e-09   + Y
SELE                 -1.499802  1.336730e-01   - Y
VCAM1                 6.710640  1.961481e-11   + Y
